# Phase 5 Analysis - PPO vs DQN Comparison
## January 2026

This notebook compares the performance of:
1. **Random Baseline** (Phase 2) - Exp 15
2. **DQN Agent** (Phase 3) - Exp 14  
3. **PPO v2 Agent** (Phase 5) - Exp 35

### Key Questions:
- How does PPO with reward shaping compare to DQN?
- Why did PPO have lower average distance despite more episodes?
- What hyperparameter changes might help?

In [ ]:
# Data manipulation
import pandas as pd

# Database connection
import psycopg2

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio
pio.renderers.default = 'notebook'

print("✅ Imports loaded successfully!")

## 1. Load Experiment Data

Loading episode metrics from PostgreSQL for all three experiments.

In [ ]:
# Database connection
conn = psycopg2.connect(
    host="localhost",
    database="mario_rl_db",
    user="mario_rl_user",
    password="Bingbongbing123!"
)

print("✅ Connected to database")

In [ ]:
# View all experiments
query = """
SELECT experiment_id, experiment_name, algorithm, start_timestamp, total_episodes
FROM experiments
ORDER BY start_timestamp;
"""

experiments_df = pd.read_sql(query, conn)
print(experiments_df)

In [ ]:
# Load episodes for all three experiments
query_random = "SELECT * FROM episodes WHERE experiment_id = 15 ORDER BY episode_number;"
query_dqn = "SELECT * FROM episodes WHERE experiment_id = 14 ORDER BY episode_number;"
query_ppo = "SELECT * FROM episodes WHERE experiment_id = 35 ORDER BY episode_number;"

random_df = pd.read_sql(query_random, conn)
dqn_df = pd.read_sql(query_dqn, conn)
ppo_df = pd.read_sql(query_ppo, conn)

print(f"Random baseline: {len(random_df)} episodes")
print(f"DQN training: {len(dqn_df)} episodes")
print(f"PPO v2 training: {len(ppo_df)} episodes")

## 2. Summary Statistics

Comparing all three agents across key metrics.

In [ ]:
# Calculate summary statistics
def get_stats(df, name):
    return {
        'Agent': name,
        'Episodes': len(df),
        'Avg Reward': df['reward'].mean(),
        'Avg Distance': df['distance_traveled'].mean(),
        'Max Distance': df['distance_traveled'].max(),
        'Avg Score': df['score'].mean(),
        'Success Rate %': (df['level_completed'].sum() / len(df)) * 100 if len(df) > 0 else 0
    }

summary = pd.DataFrame([
    get_stats(random_df, 'Random'),
    get_stats(dqn_df, 'DQN'),
    get_stats(ppo_df, 'PPO v2')
])

print("\n" + "="*70)
print("AGENT PERFORMANCE COMPARISON")
print("="*70)
print(summary.round(2).to_string(index=False))
print("="*70)

In [ ]:
# Calculate improvements over random baseline
random_avg_dist = random_df['distance_traveled'].mean()
random_avg_reward = random_df['reward'].mean()

print("\n" + "="*50)
print("IMPROVEMENT OVER RANDOM BASELINE")
print("="*50)
print("\nDQN:")
print(f"  Distance: {dqn_df['distance_traveled'].mean() / random_avg_dist:.2f}x")
print(f"  Reward: {dqn_df['reward'].mean() / random_avg_reward:.2f}x")
print("\nPPO v2:")
print(f"  Distance: {ppo_df['distance_traveled'].mean() / random_avg_dist:.2f}x")
print(f"  Reward: {ppo_df['reward'].mean() / random_avg_reward:.2f}x")
print("\nDQN vs PPO:")
print(f"  DQN avg distance: {dqn_df['distance_traveled'].mean():.1f}")
print(f"  PPO avg distance: {ppo_df['distance_traveled'].mean():.1f}")
print(f"  DQN is {dqn_df['distance_traveled'].mean() / ppo_df['distance_traveled'].mean():.2f}x better on average")

## 3. Visualizations

### 3.1 Distance Distribution Comparison

In [ ]:
# Box plot comparing all three agents
fig = go.Figure()

fig.add_trace(go.Box(
    y=random_df['distance_traveled'],
    name='Random',
    marker_color='coral',
    line=dict(width=2)
))

fig.add_trace(go.Box(
    y=dqn_df['distance_traveled'],
    name='DQN',
    marker_color='mediumseagreen',
    line=dict(width=2)
))

fig.add_trace(go.Box(
    y=ppo_df['distance_traveled'],
    name='PPO v2',
    marker_color='steelblue',
    line=dict(width=2)
))

fig.update_layout(
    title='Distance Distribution: Random vs DQN vs PPO',
    yaxis_title='Distance Traveled (pixels)',
    showlegend=True,
    height=500,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()

### 3.2 Learning Curves

In [ ]:
# DQN Learning Curve
fig = go.Figure()

# Episode distances
fig.add_trace(go.Scatter(
    x=dqn_df['episode_number'],
    y=dqn_df['distance_traveled'],
    mode='markers',
    name='Episode Distance',
    marker=dict(size=4, color='mediumseagreen'),
    opacity=0.5
))

# Rolling average
window = 50
dqn_df['dist_ma'] = dqn_df['distance_traveled'].rolling(window=window, min_periods=1).mean()

fig.add_trace(go.Scatter(
    x=dqn_df['episode_number'],
    y=dqn_df['dist_ma'],
    mode='lines',
    name=f'{window}-Episode Moving Avg',
    line=dict(color='darkgreen', width=3)
))

fig.update_layout(
    title='DQN Learning Curve: Distance Over Training',
    xaxis_title='Episode Number',
    yaxis_title='Distance (pixels)',
    height=400,
    plot_bgcolor='white'
)

fig.show()

In [ ]:
# PPO Learning Curve
fig = go.Figure()

# Episode distances
fig.add_trace(go.Scatter(
    x=ppo_df['episode_number'],
    y=ppo_df['distance_traveled'],
    mode='markers',
    name='Episode Distance',
    marker=dict(size=4, color='steelblue'),
    opacity=0.5
))

# Rolling average
window = 100
ppo_df['dist_ma'] = ppo_df['distance_traveled'].rolling(window=window, min_periods=1).mean()

fig.add_trace(go.Scatter(
    x=ppo_df['episode_number'],
    y=ppo_df['dist_ma'],
    mode='lines',
    name=f'{window}-Episode Moving Avg',
    line=dict(color='darkblue', width=3)
))

# Add milestone markers
milestones = [650, 900, 1200, 1600, 2000]
for m in milestones:
    fig.add_hline(y=m, line_dash='dot', line_color='gray', opacity=0.5)

fig.update_layout(
    title='PPO v2 Learning Curve: Distance Over Training',
    xaxis_title='Episode Number',
    yaxis_title='Distance (pixels)',
    height=400,
    plot_bgcolor='white'
)

fig.show()

### 3.3 Side-by-Side Learning Comparison

In [ ]:
# Compare learning curves side by side
fig = make_subplots(rows=1, cols=2, subplot_titles=['DQN (785 episodes)', 'PPO v2 (2197 episodes)'])

# DQN
fig.add_trace(go.Scatter(
    x=dqn_df['episode_number'],
    y=dqn_df['dist_ma'],
    mode='lines',
    name='DQN 50-ep MA',
    line=dict(color='mediumseagreen', width=2)
), row=1, col=1)

# PPO
fig.add_trace(go.Scatter(
    x=ppo_df['episode_number'],
    y=ppo_df['dist_ma'],
    mode='lines',
    name='PPO 100-ep MA',
    line=dict(color='steelblue', width=2)
), row=1, col=2)

fig.update_layout(
    title='Learning Curves Comparison',
    height=400,
    showlegend=True,
    plot_bgcolor='white'
)

# Same y-axis scale for fair comparison
fig.update_yaxes(range=[0, 2500], title_text='Distance (pixels)')
fig.update_xaxes(title_text='Episode Number')

fig.show()

### 3.4 Distance Distribution Histogram

In [ ]:
# Histogram of distances
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=dqn_df['distance_traveled'],
    name='DQN',
    opacity=0.6,
    marker_color='mediumseagreen',
    nbinsx=50
))

fig.add_trace(go.Histogram(
    x=ppo_df['distance_traveled'],
    name='PPO v2',
    opacity=0.6,
    marker_color='steelblue',
    nbinsx=50
))

fig.update_layout(
    title='Distance Distribution: DQN vs PPO',
    xaxis_title='Distance (pixels)',
    yaxis_title='Count',
    barmode='overlay',
    height=400,
    plot_bgcolor='white'
)

fig.show()

## 4. Analysis: Why DQN Outperformed PPO

Let's investigate the performance difference.

In [ ]:
# Episode length analysis
print("EPISODE LENGTH ANALYSIS")
print("="*50)
print("\nDQN:")
print(f"  Avg episode length: {dqn_df['episode_length'].mean():.1f} steps")
print(f"  Min: {dqn_df['episode_length'].min()}, Max: {dqn_df['episode_length'].max()}")
print("\nPPO v2:")
print(f"  Avg episode length: {ppo_df['episode_length'].mean():.1f} steps")
print(f"  Min: {ppo_df['episode_length'].min()}, Max: {ppo_df['episode_length'].max()}")

# PPO has early termination at 150 stuck steps
short_episodes = len(ppo_df[ppo_df['episode_length'] < 200])
print(f"\nPPO episodes < 200 steps (early terminated): {short_episodes} ({100*short_episodes/len(ppo_df):.1f}%)")

In [ ]:
# Where did agents get stuck?
print("\nDISTANCE PERCENTILES")
print("="*50)
for name, df in [('DQN', dqn_df), ('PPO', ppo_df)]:
    print(f"\n{name}:")
    for p in [25, 50, 75, 90, 95]:
        val = df['distance_traveled'].quantile(p/100)
        print(f"  {p}th percentile: {val:.0f} pixels")

In [ ]:
# Performance in last 25% of training
print("\nLATE TRAINING PERFORMANCE (Last 25%)")
print("="*50)

dqn_late = dqn_df.tail(int(len(dqn_df) * 0.25))
ppo_late = ppo_df.tail(int(len(ppo_df) * 0.25))

print(f"\nDQN (last {len(dqn_late)} episodes):")
print(f"  Avg distance: {dqn_late['distance_traveled'].mean():.1f}")
print(f"  Max distance: {dqn_late['distance_traveled'].max():.1f}")

print(f"\nPPO (last {len(ppo_late)} episodes):")
print(f"  Avg distance: {ppo_late['distance_traveled'].mean():.1f}")
print(f"  Max distance: {ppo_late['distance_traveled'].max():.1f}")

## 5. Conclusions & Next Steps

### Key Findings:

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║                     KEY FINDINGS                                  ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  1. DQN OUTPERFORMED PPO                                         ║
║     - DQN avg distance: ~1024 pixels                             ║
║     - PPO avg distance: ~688 pixels                              ║
║     - DQN reached further (max 2743 vs 2226)                     ║
║                                                                   ║
║  2. EARLY TERMINATION IMPACT                                     ║
║     - PPO had 3x more episodes (faster feedback loop)            ║
║     - But many episodes terminated early due to stuck detection  ║
║     - Agent learned to get stuck faster, not to progress further ║
║                                                                   ║
║  3. REWARD SHAPING CHALLENGES                                    ║
║     - Milestone bonuses may be too sparse                        ║
║     - Early termination penalty may be counterproductive         ║
║     - PPO needs denser reward signals                            ║
║                                                                   ║
║  4. ALGORITHM DIFFERENCES                                        ║
║     - DQN's replay buffer remembers rare successes               ║
║     - PPO discards data after each update (on-policy)            ║
║     - DQN's epsilon-greedy randomly stumbles past obstacles      ║
║                                                                   ║
╠══════════════════════════════════════════════════════════════════╣
║                     NEXT STEPS                                    ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  □ Increase max_stuck_steps (150 → 250-300)                      ║
║  □ Add more granular milestones (every 100 pixels)               ║
║  □ Lower learning rate further (0.00003 → 0.00001)               ║
║  □ Try longer training run (5M steps)                            ║
║  □ Consider curriculum learning (easier → harder)                ║
║                                                                   ║
╚══════════════════════════════════════════════════════════════════╝
""")

In [ ]:
# Close database connection
conn.close()
print("\n✅ Database connection closed")